In [1]:
import os
import cv2
import random
import numpy as np
from tqdm import tqdm

def augment_image(image):
    # Apply random transformations
    transform_type = random.choice(['flip', 'rotate', 'scale', 'brightness'])

    if transform_type == 'flip':
        image = cv2.flip(image, 1)  # Horizontal flip
    elif transform_type == 'rotate':
        angle = random.randint(-15, 15)
        h, w = image.shape[:2]
        M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
        image = cv2.warpAffine(image, M, (w, h))
    elif transform_type == 'scale':
        fx = random.uniform(0.9, 1.1)
        fy = random.uniform(0.9, 1.1)
        image = cv2.resize(image, None, fx=fx, fy=fy)
    elif transform_type == 'brightness':
        value = random.randint(-30, 30)
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        hsv[:, :, 2] = np.clip(hsv[:, :, 2] + value, 0, 255)
        image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    return image

def generate_augmented_images(class_dir, target_count):
    images = os.listdir(class_dir)
    current_count = len(images)
    image_idx = 0

    while current_count < target_count:
        img_name = images[image_idx % len(images)]
        img_path = os.path.join(class_dir, img_name)
        image = cv2.imread(img_path)
        if image is None:
            continue

        augmented = augment_image(image)
        new_filename = f"aug_{current_count}.jpg"
        cv2.imwrite(os.path.join(class_dir, new_filename), augmented)
        current_count += 1
        image_idx += 1

# Set your dataset path
dataset_path = "dataset"
target_images_per_class = 2000

for class_name in ["fall_down", "not_fallen"]:
    print(f"Augmenting {class_name}...")
    class_dir = os.path.join(dataset_path, class_name)
    generate_augmented_images(class_dir, target_images_per_class)

print("Dataset balanced to 2000 images per class.")


Augmenting fall_down...
Augmenting not_fallen...
Dataset balanced to 2000 images per class.


In [2]:
import os
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Set MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)

# Dataset path
dataset_dir = "dataset"
classes = ["fall_down", "not_fallen"]

# Prepare data
X_data = []
y_data = []

print("Extracting pose landmarks from images...")
for label, class_name in enumerate(classes):
    class_dir = os.path.join(dataset_dir, class_name)
    for img_name in tqdm(os.listdir(class_dir), desc=f"Processing {class_name}"):
        img_path = os.path.join(class_dir, img_name)
        image = cv2.imread(img_path)

        if image is None:
            continue

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            landmarks = []
            for lm in results.pose_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])  # 33 landmarks × 3 = 99 features
            X_data.append(landmarks)
            y_data.append(label)

# Convert to NumPy arrays
X = np.array(X_data)
y = np.array(y_data)

# Reshape for Conv1D input
X = X.reshape(-1, 99, 1)

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define CNN model
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(99, 1)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
print("Training CNN model...")
model.fit(x_train, y_train, epochs=15, batch_size=32, validation_data=(x_test, y_test))

# Save model
model.save("falldown_cnn_model.h5")
print("Model saved as 'falldown_cnn_model.h5'")


Extracting pose landmarks from images...


Processing not_fallen: 100%|███████████████████████████████████████████████████████| 2000/2000 [06:58<00:00,  4.78it/s]


Training CNN model...
Epoch 1/15
86/86 [==============================] - 6s 39ms/step - loss: 0.3140 - accuracy: 0.8743 - val_loss: 0.2544 - val_accuracy: 0.9075
Epoch 2/15
86/86 [==============================] - 3s 34ms/step - loss: 0.2417 - accuracy: 0.9169 - val_loss: 0.2442 - val_accuracy: 0.9163
Epoch 3/15
86/86 [==============================] - 3s 31ms/step - loss: 0.2273 - accuracy: 0.9191 - val_loss: 0.2694 - val_accuracy: 0.9031
Epoch 4/15
86/86 [==============================] - 3s 33ms/step - loss: 0.2142 - accuracy: 0.9305 - val_loss: 0.2588 - val_accuracy: 0.9090
Epoch 5/15
86/86 [==============================] - 3s 34ms/step - loss: 0.2016 - accuracy: 0.9375 - val_loss: 0.2094 - val_accuracy: 0.9280
Epoch 6/15
86/86 [==============================] - 3s 32ms/step - loss: 0.1863 - accuracy: 0.9364 - val_loss: 0.2557 - val_accuracy: 0.9075
Epoch 7/15
86/86 [==============================] - 3s 35ms/step - loss: 0.1839 - accuracy: 0.9401 - val_loss: 0.1789 - val_accuracy